# Obteniendo los datos de Lautaro Martinez en la copa américa 2024

Empiezo por importar los módulos que se verán involucrados en esta parte:

In [2]:
from statsbombpy import sb
from scripts.paths import crear_funcion_directorio
from scripts.scrapear_fbref import obtener_df_jugadores
from scripts.config import iniciar_notebook
iniciar_notebook()

## Obteniendo la data de eventos

Primero obtengo los datos de competiciones disponibles en statsbomb:

In [3]:
df_competiciones = sb.competitions()

Ahora miro su estructura y los nombres de las competiciones que están disponibles:

In [4]:
df_competiciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 74 entries, 0 to 73
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   competition_id             74 non-null     int64 
 1   season_id                  74 non-null     int64 
 2   country_name               74 non-null     object
 3   competition_name           74 non-null     object
 4   competition_gender         74 non-null     object
 5   competition_youth          74 non-null     bool  
 6   competition_international  74 non-null     bool  
 7   season_name                74 non-null     object
 8   match_updated              74 non-null     object
 9   match_updated_360          56 non-null     object
 10  match_available_360        10 non-null     object
 11  match_available            74 non-null     object
dtypes: bool(2), int64(2), object(8)
memory usage: 6.1+ KB


In [5]:
df_competiciones.competition_name.value_counts()

competition_name
Champions League           18
La Liga                    18
FIFA World Cup              8
Copa del Rey                3
Ligue 1                     3
FA Women's Super League     3
UEFA Euro                   2
1. Bundesliga               2
Women's World Cup           2
Liga Profesional            2
Serie A                     2
Premier League              2
African Cup of Nations      1
Copa America                1
Major League Soccer         1
FIFA U20 World Cup          1
Indian Super league         1
North American League       1
NWSL                        1
UEFA Europa League          1
UEFA Women's Euro           1
Name: count, dtype: int64

Me quedo solo con la copa américa:

In [6]:
df_competiciones[df_competiciones['competition_name'] == 'Copa America']

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
21,223,282,South America,Copa America,male,False,True,2024,2024-07-15T18:00:33.653673,None,None,2024-07-15T18:00:33.653673


Ahora que conozco su id de competición e id de temporada puedo obtener todos sus datos:

In [7]:
datos_copa_america = sb.matches(competition_id=223, season_id=282).sort_values(by='match_date')

Del mismo modo paso a ver la estructura:

In [8]:
datos_copa_america.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32 entries, 31 to 0
Data columns (total 22 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   match_id               32 non-null     int64 
 1   match_date             32 non-null     object
 2   kick_off               32 non-null     object
 3   competition            32 non-null     object
 4   season                 32 non-null     object
 5   home_team              32 non-null     object
 6   away_team              32 non-null     object
 7   home_score             32 non-null     int64 
 8   away_score             32 non-null     int64 
 9   match_status           32 non-null     object
 10  match_status_360       32 non-null     object
 11  last_updated           32 non-null     object
 12  last_updated_360       0 non-null      object
 13  match_week             32 non-null     int64 
 14  competition_stage      32 non-null     object
 15  stadium                32 non-

In [9]:
datos_copa_america.head()

,match_id,match_date,kick_off,competition,season,home_team,away_team,home_score,away_score,match_status,match_status_360,last_updated,last_updated_360,match_week,competition_stage,stadium,referee,home_managers,away_managers,data_version,shot_fidelity_version,xy_fidelity_version
31,3939969,2024-06-21,03:00:00.000,South America - Copa America,2024,Argentina,Canada,2,0,available,unscheduled,2024-07-09T10:26:47.528559,None,1,Group Stage,Mercedes-Benz Stadium,Jesús Valenzuela Sáez,Lionel Sebastián Scaloni,Jesse Marsch,1.1.0,2,2
30,3939970,2024-06-22,03:00:00.000,South America - Copa America,2024,Peru,Chile,0,0,available,unscheduled,2024-07-07T15:51:25.411244,None,1,Group Stage,AT&T Stadium,Wilton Pereira Sampaio,Jorge Daniel Fossati Lurachi,Ricardo Alberto Gareca Nardi,1.1.0,2,2
29,3939971,2024-06-23,04:00:00.000,South America - Copa America,2024,Mexico,Jamaica,1,0,available,unscheduled,2024-07-12T19:15:02.637961,None,1,Group Stage,NRG Stadium,Ismail Elfath,Jaime Arturo Lozano Espín,Heimir Hallgrímsson,1.1.0,2,2
28,3939972,2024-06-23,01:00:00.000,South America - Copa America,2024,Ecuador,Venezuela,1,2,available,unscheduled,2024-07-12T21:31:59.102963,None,1,Group Stage,Levi''s Stadium,Wilmar Alexander Roldán Pérez,Félix Sánchez Bas,Fernando Ariel Batista,1.1.0,2,2
27,3939974,2024-06-24,01:00:00.000,South America - Copa America,2024,United States,Bolivia,2,0,available,unscheduled,2024-07-10T13:06:41.582507,None,1,Group Stage,AT&T Stadium,Maurizio Mariani,Gregg Berhalter,Antônio Carlos Zago,1.1.0,2,2


Como el foco de este proyecto está en Lautaro Martinez, jugador de la selección Argentina, entonces procedo a quedarme solo con los partidos de dicha selección:

In [10]:
df_partidos_argentina = datos_copa_america[(datos_copa_america['home_team'] == 'Argentina') | (datos_copa_america['away_team'] == 'Argentina')]

Ahora lo ideal sería ver que contiene la data de eventos para un partido dado, por ejemplo voy a analizar que eventos se midieron para el primer partido de la selección Argentina:

In [21]:
sb.events(match_id=df_partidos_argentina.head(1)['match_id'].values[0], split=True).keys()

dict_keys(['starting_xis', 'half_starts', 'passes', 'ball_receipts', 'carrys', 'pressures', 'blocks', 'dribbled_pasts', 'dribbles', 'ball_recoverys', 'shots', 'goal_keepers', 'clearances', 'foul_committeds', 'miscontrols', 'dispossesseds', 'duels', 'foul_wons', 'interceptions', 'half_ends', '50/50s', 'substitutions', 'tactical_shifts', 'shields', 'injury_stoppages', 'referee_ball_drops', 'player_offs', 'player_ons'])

Para efectos de analizar a Lautaro la data de eventos que me interesa analizar se encuentra en los atributos **'shots', 'foul_wons', 'ball_receipts' y 'passes'**. Por lo tanto cuando llegue la hora de guardar todos estos datos solo voy a considerar esos eventos.

## Obteniendo la data de estadísticas varias

In [30]:
df_stats_standard = obtener_df_jugadores('https://fbref.com/es/comps/685/stats/Estadisticas-de-Copa-America')
df_stats_shooting = obtener_df_jugadores('https://fbref.com/es/comps/685/shooting/Estadisticas-de-Copa-America')
df_stats_gca = obtener_df_jugadores('https://fbref.com/es/comps/685/gca/Estadisticas-de-Copa-America')

## Guardando los datos

In [31]:
proyectos_dir = crear_funcion_directorio('proyectos')

df_stats_standard.to_csv(proyectos_dir('ficha_lautaro_martinez_CA2024','data' ,'raw', 'df_stats_standard.csv'), index=False)
df_stats_shooting.to_csv(proyectos_dir('ficha_lautaro_martinez_CA2024','data' ,'raw', 'df_stats_shooting.csv'), index=False)
df_stats_gca.to_csv(proyectos_dir('ficha_lautaro_martinez_CA2024','data' ,'raw', 'df_stats_gca.csv'), index=False)

In [32]:
atributos_interes = ['shots', 'foul_wons', 'ball_receipts', 'passes']

for i, partido in df_partidos_argentina.iterrows():
    df_eventos = sb.events(match_id=partido['match_id'], split=True)
    for atributo in atributos_interes:
        df_eventos[atributo].to_csv(proyectos_dir('ficha_lautaro_martinez_CA2024','data' ,'raw', f'df_eventos_{partido["match_id"]}_{atributo}.csv'), index=False)